## Imports

In [1]:
import numpy as np
import pandas as pd
from lmfit.models import Model
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
from pandas.api.types import CategoricalDtype

## Pupillary Functions


Task evoked pupillary response is calculated after correcting for luminance-induced pupil dilation: $𝑇𝐸𝑃𝑅 = 𝑑_m − 𝑑(𝑌)$, where $d_m$ is the measured pupil dilation, and $d(Y)$ is the predicted pupil dilation for the given luminance level. 

Predicted pupil dilation is calculated from a calibration sequence that produces and individual mapping model for each participant. The calibration sequence consists of 8 solid gray colors with varying luminance levels displayed in a psuedo-random order for 6 seconds each. The luminance levels span the range from 0.0 to 0.78, and for each calibration level, the first 0.5s of data is discarded to account for the initial pupillary response to the change in luminance, which can take a maximum of 0.5s. . The individual mapping model is calculated using a non-linear least squares regression to fit the equation $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$ to the measured pupil dilation data for each participant. 

Pupil dilation data and the average luminance data were collected at 90 Hz, the display rate of the HMD.

See: Eckert, M., Robotham, T., Habets, E. A. P., and Rummukainen, O. S. (2022). Pupillary Light Reflex Correction for Robust Pupillometry in Virtual Reality. Proc. ACM Comput. Graph. Interact. Tech. 5, 1–16. doi: 10.1145/3530798

In [2]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [3]:
def modmax(d):
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])
    # if value is larger than both neighbours , and strictly larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(len(d)):
        ll = m[i -1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
        # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    return t

In [4]:
def ipa_func(d):
    # obtain 2-level DWT of pupil diameter signal d
    try:
        (cA2 ,cD2 ,cD1) = pywt.wavedec(d,'sym16', 'per', level=2)
    except ValueError :
        return
    # get signal duration (in seconds)
    tt = d.index[-1] - d.index[0]
    # normalize by 1/2 j , j = 2 for 2-level DWT
    cA2 [:] = [x / math.sqrt (4.0) for x in cA2]
    cD1 [:] = [x / math.sqrt (2.0) for x in cD1]
    cD2 [:] = [x / math.sqrt (4.0) for x in cD2]

    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    # threshold using universal threshold λuniv = σˆp(2logn)
    # where σˆ is the standard deviation of the noise
    λuniv = np.std(cD2m) * math.sqrt (2.0* np.log2(len(cD2m )))
    cD2t = pywt. threshold (cD2m ,λuniv,mode="hard")
    # compute IPA
    ctr = 0
    for i in range(len(cD2t )):
        if math.fabs(cD2t[i]) > 0: ctr += 1
    IPA = float(ctr)/tt.total_seconds()

    return IPA

## Statistical Functions

In [5]:
def iqr_outlier_indices(data):
    q1 = data.quantile(.25)
    q3 = data.quantile(.75)
    iqr = stats.iqr(data, nan_policy='omit', rng=(25, 75))
    return np.where((data < (q1 - 1.5 * iqr)) | (data > (q3 + 1.5 * iqr)))

In [6]:
def iqr_stats(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = stats.iqr(data, nan_policy='omit', rng=(25, 75))
    return iqr, q1, q3

In [7]:
def get_results_colors(np, wp, tp):
    pastels = px.colors.qualitative.Pastel2
    default_color = 'white'
    significant_color = pastels[0]
    non_significant_color = pastels[3]

    normal_color = non_significant_color if np < 0.05 else significant_color
    wilcox_color = default_color
    ttest_color = default_color
    if np < 0.05:
        wilcox_color = significant_color if wp < 0.05 else non_significant_color
    else:
        ttest_color = significant_color if tp < 0.05 else non_significant_color

    fill_color = [[default_color, default_color, default_color],
                  [default_color, wilcox_color, ttest_color] , 
                  [normal_color, wilcox_color, ttest_color]]
    
    return fill_color

## Data Processing Functions

In [8]:
method_cats = CategoricalDtype(['4DoF','6DoF', 'unimanual','bimanual'], ordered=False)
model_cats = CategoricalDtype(['A', 'B', 'C', 'D'], ordered=True)
block_cats = CategoricalDtype(['0', '1', '2', '3'], ordered=True)
event_cats = CategoricalDtype(['Start', 'PointPlaced', 'Move', 'End', 'Draw', 'Erase', 'PointDeleted'], ordered=False)
target_cats = CategoricalDtype(['1','2'], ordered=False)
trial_cats = CategoricalDtype(['0','1','2','3'], ordered=True)
data_names = ['id', 'model', 'method']

## Import Data

In [9]:
user_data_nav = pd.read_pickle('user_data_nav.pkl')
user_data_crt = pd.read_pickle('user_data_crt.pkl')

## Dataframe Structure

In [10]:
user_data_nav.head()

luminance  pupilDiameter  pupil_lum_base  adj_pupil  \
id  model method                                                        
108 A     4DoF     0.224356       4.866458        3.831577   1.034881   
          6DoF     0.225511       4.359483        3.829147   0.530336   
    B     4DoF     0.242239       4.650883        3.771303   0.879580   
          6DoF     0.256185       4.437137        3.746679   0.690458   
    C     4DoF     0.223505       4.658904        3.852900   0.806004   

                  total_time       IPA  discomfort  seq  
id  model method                                         
108 A     4DoF       37.0822  0.242846           2    0  
          6DoF       61.0043  0.098389           1    0  
    B     4DoF       67.7005  0.147757           1    0  
          6DoF       96.4316  0.197076           1    0  
    C     4DoF       88.0356  0.227236           1    1

In [11]:
user_data_nav.describe()

,luminance,pupilDiameter,pupil_lum_base,adj_pupil,total_time,IPA,discomfort,seq
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000
mean,0.227208,3.697572,3.232291,0.465281,66.730551,0.174273,2.459259,0.670370
std,0.021534,0.512666,0.482426,0.293403,27.594378,0.054892,2.555737,0.865839
min,0.158152,2.448724,2.170120,-0.354323,24.187100,0.054860,0.000000,0.000000
25%,0.214954,3.304627,3.006914,0.230412,48.380099,0.130967,0.000000,0.000000
50%,0.224900,3.689174,3.196184,0.465254,57.755500,0.171791,1.000000,0.000000
75%,0.240174,4.055160,3.480494,0.694074,76.352525,0.213827,4.000000,1.000000
max,0.294793,5.205173,4.245193,1.268353,212.109400,0.342511,10.000000,4.000000


In [12]:
user_data_crt.head()

luminance  pupilDiameter  pupil_lum_base  adj_pupil  \
id  model method                                                           
108 A     unimanual   0.280843       4.402825        3.651643   0.751182   
          bimanual    0.259386       5.104381        3.717137   1.387243   
    B     unimanual   0.320817       4.189030        3.540984   0.648046   
          bimanual    0.263313       4.835740        3.714536   1.121204   
    C     unimanual   0.237598       4.375924        3.783636   0.592288   

                     total_time       IPA  seq  PointPlaced  Move  Draw  \
id  model method                                                          
108 A     unimanual     98.3030  0.111912    1           19     6     0   
          bimanual      62.7652  0.107009    2            0     0     4   
    B     unimanual     95.2104  0.136582    2           17     3     0   
          bimanual      40.4416  0.173139    2            0     0     4   
    C     unimanual     36.4409  0.247039    1           12     1     0   

                     Erase  PointDeleted  
id  model method                          
108 A     unimanual      0             0  
          bimanual       0             0  
    B     unimanual      0             0  
          bimanual       0             0  
    C     unimanual      0             0

In [13]:
user_data_crt.describe()

,luminance,pupilDiameter,pupil_lum_base,adj_pupil,total_time,IPA,seq,PointPlaced,Move,Draw,Erase,PointDeleted
count,272.000000,272.000000,272.000000,272.000000,272.000000,272.000000,272.000000,272.000000,272.000000,272.000000,272.000000,272.000000
mean,0.245325,3.760838,3.201510,0.559328,58.287357,0.177029,0.812500,5.069853,3.625000,2.566176,0.621324,0.238971
std,0.036036,0.592617,0.486131,0.328117,41.766342,0.065565,0.985948,6.891267,7.754477,6.273828,2.357368,0.827272
min,0.164554,2.419907,2.162557,-0.141567,11.261400,0.043479,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.214720,3.303979,2.946377,0.311914,32.817550,0.133438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.243208,3.776663,3.124938,0.536255,46.113950,0.171491,1.000000,2.500000,0.000000,0.500000,0.000000,0.000000
75%,0.272621,4.207483,3.486466,0.752358,70.038900,0.217677,1.000000,8.000000,2.000000,2.000000,0.000000,0.000000
max,0.336842,5.373014,4.248593,1.713309,371.058100,0.401104,4.000000,50.000000,38.000000,61.000000,19.000000,6.000000


## Statistical Analysis

### Navigation Workload

#### IPA Evaluation

In [14]:
user_data_nav

luminance  pupilDiameter  pupil_lum_base  adj_pupil  \
id  model method                                                        
108 A     4DoF     0.224356       4.866458        3.831577   1.034881   
          6DoF     0.225511       4.359483        3.829147   0.530336   
    B     4DoF     0.242239       4.650883        3.771303   0.879580   
          6DoF     0.256185       4.437137        3.746679   0.690458   
    C     4DoF     0.223505       4.658904        3.852900   0.806004   
...                     ...            ...             ...        ...   
149 B     6DoF     0.245416       4.517930        3.847571   0.670359   
    C     4DoF     0.209773       4.522400        3.910984   0.611416   
          6DoF     0.210505       4.735390        3.906888   0.828502   
    D     4DoF     0.216912       4.532409        3.873535   0.658874   
          6DoF     0.223390       4.741807        3.869410   0.872397   

                  total_time       IPA  discomfort  seq  
id  model method                                         
108 A     4DoF       37.0822  0.242846           2    0  
          6DoF       61.0043  0.098389           1    0  
    B     4DoF       67.7005  0.147757           1    0  
          6DoF       96.4316  0.197076           1    0  
    C     4DoF       88.0356  0.227236           1    1  
...                      ...       ...         ...  ...  
149 B     6DoF       75.7743  0.118807           1    2  
    C     4DoF       44.9334  0.200391           1    0  
          6DoF       71.3219  0.112203           2    2  
    D     4DoF       37.5156  0.186698           1    0  
          6DoF       50.2986  0.218790           2    3  

[270 rows x 8 columns]

In [15]:
ipa_nav = user_data_nav.loc[(slice(None), slice(None), slice(None)), 'IPA']
ipa_nav = ipa_nav.unstack(level=(2))

outliers_4dof = iqr_outlier_indices(ipa_nav['4DoF'])
ipa_nav = ipa_nav.drop(ipa_nav.iloc[outliers_4dof].index)

outliers_6dof = iqr_outlier_indices(ipa_nav['6DoF'])
ipa_nav = ipa_nav.drop(ipa_nav.iloc[outliers_6dof].index)

ipa_4dof = ipa_nav['4DoF']
ipa_6dof = ipa_nav['6DoF']

n_stat, n_p = stats.shapiro(ipa_nav.stack())
w_stat, w_p = stats.wilcoxon(ipa_4dof, ipa_6dof, nan_policy='omit')
t_stat, t_p = stats.ttest_rel(ipa_4dof, ipa_6dof, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(ipa_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(ipa_6dof)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [ipa_4dof.median(), ipa_6dof.median()], [iqr_4dof, iqr_6dof], [ipa_4dof.min(), ipa_6dof.min()], [ipa_4dof.max(), ipa_6dof.max()], [ipa_4dof.skew(), ipa_6dof.skew()]]
    

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [ipa_4dof.mean(), ipa_6dof.mean()], [ipa_4dof.std(), ipa_6dof.std()], [ipa_4dof.min(), ipa_6dof.min()], [ipa_4dof.max(), ipa_6dof.max()], [ipa_4dof.skew(), ipa_6dof.skew()]]

In [16]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

iqr_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

iqr_fig.add_trace(stats_table, row=2, col=1)
iqr_fig.add_trace(go.Box(y=ipa_nav['4DoF'], name="4-DoF", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(go.Box(y=ipa_nav['6DoF'], name="6-DoF", notched=True, notchwidth=0.15), row=1, col=2)
iqr_fig.add_trace(results_table, row=1, col=1)

iqr_fig.update_layout(
    title_text="IPA by Navigation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='IPA',
)

iqr_fig.update_yaxes(range=[0,max(1.1*ipa_4dof.max(), 1.1*ipa_6dof.max())])
iqr_fig.show()

#### TEPR Evaluation

In [17]:
tepr_nav = user_data_nav.loc[(slice(None), slice(None),  slice(None)), 'adj_pupil']
tepr_nav = tepr_nav.unstack(level=(2))

outliers_4dof = iqr_outlier_indices(tepr_nav['4DoF'])
tepr_nav = tepr_nav.drop(ipa_nav.iloc[outliers_4dof].index)

outliers_6dof = iqr_outlier_indices(tepr_nav['6DoF'])
tepr_nav = tepr_nav.drop(tepr_nav.iloc[outliers_6dof].index)

tepr_4dof = tepr_nav['4DoF']
tepr_6dof = tepr_nav['6DoF']

n_stat, n_p = stats.shapiro(tepr_nav.stack())
w_stat, w_p = stats.wilcoxon(tepr_4dof, tepr_6dof, nan_policy='omit')
t_stat, t_p = stats.ttest_rel(tepr_4dof, tepr_6dof, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(tepr_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(tepr_6dof)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], 
                      [tepr_4dof.median(), tepr_6dof.median()], 
                      [iqr_4dof, iqr_6dof], 
                      [tepr_4dof.min(), tepr_6dof.min()], 
                      [tepr_4dof.max(), tepr_6dof.max()], 
                      [tepr_4dof.skew(), tepr_6dof.skew()]]
    

else:
    header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    values = [['<b>4-DoF</b>', '<b>6-DoF</b'], 
              [ipa_4dof.mean(), tepr_6dof.mean()], 
              [ipa_4dof.std(), tepr_6dof.std()], 
              [ipa_4dof.min(), tepr_6dof.min()], 
              [ipa_4dof.max(), tepr_6dof.max()], 
              [ipa_4dof.skew(), tepr_6dof.skew()]]
    prefix=['t = ','']

In [18]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

tepr_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

tepr_fig.add_trace(stats_table, row=2, col=1)
tepr_fig.add_trace(go.Box(y=tepr_nav['4DoF'], name="4-DoF", notched=True, notchwidth=0.15), row=1, col=2)
tepr_fig.add_trace(go.Box(y=tepr_nav['6DoF'], name="6-DoF", notched=True, notchwidth=0.15), row=1, col=2)
tepr_fig.add_trace(results_table, row=1, col=1)

tepr_fig.update_layout(
    title_text="TEPR by Navigation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='TEPR',
)

tepr_max = max(tepr_4dof.max(), tepr_6dof.max())
tepr_min = min(tepr_4dof.min(), tepr_6dof.min())
tepr_range = 0.025 * (tepr_max - tepr_min)
tepr_fig.update_yaxes(range=[min(tepr_4dof.min() - tepr_range, tepr_6dof.min() - tepr_range), max(tepr_4dof.max() + tepr_range, tepr_6dof.max() + tepr_range)])
tepr_fig.show()

### Creation Workload

#### IPA Evaluation

In [19]:
ipa_crt = user_data_crt.loc[(slice(None), slice(None), slice(None)), 'IPA']
ipa_crt = ipa_crt.unstack(level=(2))

uni_outliers = iqr_outlier_indices(ipa_crt['unimanual'])
ipa_crt = ipa_crt.drop(ipa_crt.iloc[uni_outliers].index)

bi_outliers = iqr_outlier_indices(ipa_crt['bimanual'])
ipa_crt = ipa_crt.drop(ipa_crt.iloc[bi_outliers].index)

ipa_uni = ipa_crt['unimanual']
ipa_bi = ipa_crt['bimanual']

n_stat, n_p = stats.shapiro(ipa_crt.stack())
w_stat, w_p = stats.wilcoxon(ipa_uni, ipa_bi)
t_stat, t_p = stats.ttest_rel(ipa_uni, ipa_bi)

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    uni_iqr, uni_q1, uni_q3 = iqr_stats(ipa_uni)
    bi_iqr, bi_q1, bi_q3 = iqr_stats(ipa_bi)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Unimanual</b>', '<b>Bimanual</b>'], [ipa_uni.median(), ipa_bi.median()], [uni_iqr, bi_iqr], [ipa_uni.min(), ipa_bi.min()], [ipa_uni.max(), ipa_bi.max()], [ipa_uni.skew(), ipa_bi.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Unimanual</b>', '<b>Bimanual</b>'], [ipa_uni.mean(), ipa_bi.mean()], [ipa_uni.std(), ipa_bi.std()], [ipa_uni.min(), ipa_bi.min()], [ipa_uni.max(), ipa_bi.max()], [ipa_uni.skew(), ipa_bi.skew()]]


In [20]:
stats_table = go.Table(columnwidth=[5,4,4,4,4], name='Summary')
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

ipa_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

ipa_fig.add_trace(stats_table, row=2, col=1)
ipa_fig.add_trace(go.Box(y=ipa_crt['bimanual'], name="Bimanual", notched=True, notchwidth=0.15), row=1, col=2)
ipa_fig.add_trace(go.Box(y=ipa_crt['unimanual'], name="Unimanual", notched=True, notchwidth=0.15), row=1, col=2)
ipa_fig.add_trace(results_table, row=1, col=1)

ipa_fig.update_layout(
    title_text="IPA by Creation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='IPA',
)

ipa_max = max(ipa_uni.max(), ipa_bi.max())
ipa_min = min(ipa_uni.min(), ipa_bi.min())
ipa_min = 0 if ipa_min > 0 else ipa_min
ipa_range = 0.025 * (ipa_max - ipa_min)
ipa_fig.update_yaxes(range=[0,max(1.1*ipa_uni.max(), 1.1*ipa_bi.max())])
ipa_fig.show()

#### TEPR Evaluation

In [21]:
tepr_crt = user_data_crt.loc[(slice(None), slice(None), slice(None)), 'adj_pupil']
tepr_crt = tepr_crt.unstack(level=(2))

uni_outliers = iqr_outlier_indices(tepr_crt['unimanual'])
tepr_crt = tepr_crt.drop(tepr_crt.iloc[uni_outliers].index)

bi_outliers = iqr_outlier_indices(tepr_crt['bimanual'])
tepr_crt = tepr_crt.drop(tepr_crt.iloc[bi_outliers].index)

tepr_uni = tepr_crt['unimanual']
tepr_bi = tepr_crt['bimanual']

n_stat, n_p = stats.shapiro(tepr_crt.stack())
w_stat, w_p = stats.wilcoxon(tepr_uni, tepr_bi)
t_stat, t_p = stats.ttest_rel(tepr_uni, tepr_bi)

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    uni_iqr, uni_q1, uni_q3 = iqr_stats(tepr_uni)
    bi_iqr, bi_q1, bi_q3 = iqr_stats(tepr_bi)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Unimanual</b>', '<b>Bimanual</b>'], 
                      [tepr_uni.median(), tepr_bi.median()], 
                      [uni_iqr, bi_iqr], 
                      [tepr_uni.min(), tepr_bi.min()], 
                      [tepr_uni.max(), tepr_bi.max()], 
                      [tepr_uni.skew(), tepr_bi.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Unimanual</b>', '<b>Bimanual</b>'], 
                      [tepr_uni.mean(), tepr_bi.mean()], 
                      [tepr_uni.std(), tepr_bi.std()], 
                      [tepr_uni.min(), tepr_bi.min()], 
                      [tepr_uni.max(), tepr_bi.max()], 
                      [tepr_uni.skew(), tepr_bi.skew()]]

In [22]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

tepr_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

tepr_fig.add_trace(stats_table, row=2, col=1)
tepr_fig.add_trace(go.Box(y=tepr_crt['bimanual'], name="Bimanual", notched=True, notchwidth=0.15), row=1, col=2)
tepr_fig.add_trace(go.Box(y=tepr_crt['unimanual'], name="Unimanual", notched=True, notchwidth=0.15), row=1, col=2)
tepr_fig.add_trace(results_table, row=1, col=1)

tepr_fig.update_layout(
    title_text="TEPR by Creation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='TEPR',
)

tepr_max = max(tepr_uni.max(), tepr_bi.max())
tepr_min = min(tepr_uni.min(), tepr_bi.min())
tepr_range = 0.025 * (tepr_max - tepr_min)
tepr_fig.update_yaxes(range=[min(tepr_uni.min() - tepr_range, tepr_bi.min() - tepr_range), max(tepr_uni.max() + tepr_range, tepr_bi.max() + tepr_range)])
tepr_fig.show()


### Survey Analysis

N.B: 

Non-normally distributed Likert data is known to be poorly suited to non-parametric tests. Mixed models are a better alternative for this type of data for null hypothesis testing, especially for repeated measures designs. This will be refactored soon.

In [23]:
discomfort = user_data_nav.loc[(slice(None), slice(None), slice(None)), 'discomfort']
discomfort = discomfort.unstack(level=(2))

outliers_4dof = iqr_outlier_indices(discomfort['4DoF'])
discomfort = discomfort.drop(discomfort.iloc[outliers_4dof].index)

outliers_6dof = iqr_outlier_indices(discomfort['6DoF'])
discomfort = discomfort.drop(discomfort.iloc[outliers_6dof].index)

discomfort_4dof = discomfort['4DoF']
discomfort_6dof = discomfort['6DoF']

n_stat, n_p = stats.shapiro(discomfort.stack())
w_stat, w_p = stats.wilcoxon(discomfort_4dof, discomfort_6dof, nan_policy='omit', zero_method='pratt')
t_stat, t_p = stats.ttest_rel(discomfort_4dof, discomfort_6dof, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors 

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(discomfort_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(discomfort_6dof)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [discomfort_4dof.median(), discomfort_6dof.median()], [iqr_4dof, iqr_6dof], [discomfort_4dof.min(), discomfort_6dof.min()], [discomfort_4dof.max(), discomfort_6dof.max()], [discomfort_4dof.skew(), discomfort_6dof.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [discomfort_4dof.mean(), discomfort_6dof.mean()], [discomfort_4dof.std(), discomfort_6dof.std()], [discomfort_4dof.min(), discomfort_6dof.min()], [discomfort_4dof.max(), discomfort_6dof.max()], [discomfort_4dof.skew(), discomfort_6dof.skew()]]

In [24]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

discomfort_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

discomfort_fig.add_trace(stats_table, row=2, col=1)
discomfort_fig.add_trace(go.Box(y=discomfort['4DoF'], name="4-DoF", notched=True, notchwidth=0.15), row=1, col=2)
discomfort_fig.add_trace(go.Box(y=discomfort['6DoF'], name="6-DoF", notched=True, notchwidth=0.15), row=1, col=2)
discomfort_fig.add_trace(results_table, row=1, col=1)

discomfort_fig.update_layout(
    title_text="Discomfort by Navigation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='Discomfort',
)

discomfort_fig.update_yaxes(range=[0,max(1.1*discomfort_4dof.max(), 1.1*discomfort_6dof.max())])
discomfort_fig.show()

In [25]:
seq_nav = user_data_nav.loc[(slice(None), slice(None), slice(None)), 'seq']
seq_nav = seq_nav.unstack(level=(2))

outliers_4dof = iqr_outlier_indices(seq_nav['4DoF'])
seq_nav = seq_nav.drop(seq_nav.iloc[outliers_4dof].index)

outliers_6dof = iqr_outlier_indices(seq_nav['6DoF'])
seq_nav = seq_nav.drop(seq_nav.iloc[outliers_6dof].index)

seq_4dof = seq_nav['4DoF']
seq_6dof = seq_nav['6DoF']

n_stat, n_p = stats.shapiro(seq_nav.stack())
w_stat, w_p = stats.wilcoxon(seq_4dof, seq_6dof, nan_policy='omit', zero_method='pratt')
t_stat, t_p = stats.ttest_rel(seq_4dof, seq_6dof, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    iqr_4dof, q1_4dof, q3_4dof = iqr_stats(seq_4dof)
    iqr_6dof, q1_6dof, q3_6dof = iqr_stats(seq_6dof)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [seq_4dof.median(), seq_6dof.median()], [iqr_4dof, iqr_6dof], [seq_4dof.min(), seq_6dof.min()], [seq_4dof.max(), seq_6dof.max()], [seq_4dof.skew(), seq_6dof.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>4-DoF</b>', '<b>6-DoF</b>'], [seq_4dof.mean(), seq_6dof.mean()], [seq_4dof.std(), seq_6dof.std()], [seq_4dof.min(), seq_6dof.min()], [seq_4dof.max(), seq_6dof.max()], [seq_4dof.skew(), seq_6dof.skew()]]

In [26]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

seq_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

seq_fig.add_trace(stats_table, row=2, col=1)
seq_fig.add_trace(go.Box(y=seq_nav['4DoF'], name="4-DoF", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(go.Box(y=seq_nav['6DoF'], name="6-DoF", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(results_table, row=1, col=1)

seq_fig.update_layout(
    title_text="SEQ by Navigation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='SEQ',
)

seq_fig.update_yaxes(range=[0,max(1.1*seq_4dof.max(), 1.1*seq_6dof.max())])
seq_fig.show()


In [27]:
seq_crt = user_data_crt.loc[(slice(None), slice(None), slice(None)), 'seq']
seq_crt = seq_crt.unstack(level=(2))

outliers_uni = iqr_outlier_indices(seq_crt['unimanual'])
seq_crt = seq_crt.drop(seq_crt.iloc[outliers_uni].index)

outliers_bi = iqr_outlier_indices(seq_crt['bimanual'])
seq_crt = seq_crt.drop(seq_crt.iloc[outliers_bi].index)

seq_uni = seq_crt['unimanual']
seq_bi = seq_crt['bimanual']

n_stat, n_p = stats.shapiro(seq_crt.stack())
w_stat, w_p = stats.wilcoxon(seq_uni, seq_bi, nan_policy='omit', zero_method='pratt')
t_stat, t_p = stats.ttest_rel(seq_uni, seq_bi, nan_policy='omit')

results_colors = get_results_colors(n_p, w_p, t_p)
results_header = ['<b>Test</b>', '<b>Result</b>', '<b>p-value</b>']
results_values=[['Shapiro-Wilk', 'Wilcoxon', 'T-test'], [n_stat, w_stat, t_stat], [n_p, w_p, t_p]]
results_prefix = ['','', 'p = ',]
fill_color = results_colors

if n_p < 0.05:
    iqr_uni, q1_uni, q3_uni = iqr_stats(seq_uni)
    iqr_bi, q1_bi, q3_bi = iqr_stats(seq_bi)
    summary_header = ['', '<b>Median</b>', '<b>IQR</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Unimanual</b>', '<b>Bimanual</b>'], [seq_uni.median(), seq_bi.median()], [iqr_uni, iqr_bi], [seq_uni.min(), seq_bi.min()], [seq_uni.max(), seq_bi.max()], [seq_uni.skew(), seq_bi.skew()]]

else:
    summary_header = ['', '<b>Mean</b>', '<b>STD</b>', '<b>Min</b>', '<b>Max</b>', '<b>Skewness</b>']
    summary_values = [['<b>Unimanual</b>', '<b>Bimanual</b>'], [seq_uni.mean(), seq_bi.mean()], [seq_uni.std(), seq_bi.std()], [seq_uni.min(), seq_bi.min()], [seq_uni.max(), seq_bi.max()], [seq_uni.skew(), seq_bi.skew()]]
    


In [28]:
stats_table = go.Table(columnwidth=[5,4,4,4,4])
stats_table.header = dict(
    values=summary_header, 
    align = ['right', 'center'],)
stats_table.cells = dict(
        values=summary_values,
        align = ['right', 'center'],
        format=["", ".3f"],
        )

results_table = go.Table(columnwidth=[5,4,4])
results_table.header = dict(
    values= results_header, 
    align = ['right', 'center'],)
results_table.cells = dict(
        values=results_values,
        align = ['right', 'center'],
        format=["", ".2f", ".5f"],
        fill_color=fill_color,
        prefix=results_prefix,
        )

seq_fig = make_subplots(
    rows=2, cols=2,
    shared_xaxes=False,
    specs=[
        [{"type": "table"}, {"type": "box", "rowspan": 2}],
        [{"type": "table"}, None]
    ])

seq_fig.add_trace(stats_table, row=2, col=1)
seq_fig.add_trace(go.Box(y=seq_crt['bimanual'], name="Bimanual", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(go.Box(y=seq_crt['unimanual'], name="Unimanual", notched=True, notchwidth=0.15), row=1, col=2)
seq_fig.add_trace(results_table, row=1, col=1)

seq_fig.update_layout(
    title_text="SEQ by Creation Method",
    width=1200,
    xaxis_title='Method',
    yaxis_title='SEQ',
)

seq_fig.update_yaxes(range=[0,7])
seq_fig.show()

